## Importing Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import numpy as np
import os

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
from keras import applications


import torch
import torchvision
import torchvision.transforms as transform
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, ConcatDataset
import torch.nn as nn
import torchvision.models as models
from torchvision.utils import make_grid
import torch.nn.functional as F
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras.preprocessing.image import ImageDataGenerator



from keras import backend as K

from tensorflow.keras.utils import to_categorical


from sklearn.model_selection import train_test_split

## Loading the Dataset

In [ ]:
labels = ['dandelion', 'daisy','tulip','sunflower','rose']
img_size = 224
train_transform = transform.Compose([
    transform.Resize((img_size, img_size)),
    transform.ToTensor()])
train_dataset = ImageFolder('../input/ee379v-hw4/train', transform=train_transform)

## Data Visualization and Preprocessing

In [ ]:
l = []
for i in train_dataset:
    l.append(i[1])


Random 10 images

In [ ]:
X = []
for i in train_dataset:
    X.append(np.array(i[0]))
    

In [ ]:
X=np.array(X)
l=np.array(l)
X=X/255


In [ ]:
print(X.shape)

## Create your own validation set

In [ ]:
my_train, my_val = train_test_split(X, test_size=0.1, shuffle=True, random_state=42)
y_train, y_val = train_test_split(l, test_size=0.1, shuffle=True, random_state=42)







In [ ]:
def create_model_from_scratch():

  
    model = Sequential()
    model.add(Conv2D(28, (3,3), padding='same', input_shape=my_train.shape[1:], activation='relu', name='conv_1'))
    model.add(Conv2D(28, (3,3), padding='same',activation='relu', name='conv_2'))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same', name='maxpool_1'))
    model.add(Dropout(0.25))

    model.add(Conv2D(56, (3,3), padding='same', activation='relu', name='conv_3'))
    model.add(Conv2D(56, (3,3), padding='same',activation='relu', name='conv_4'))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same', name='maxpool_2'))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(112, (3,3), padding='same', activation='relu', name='conv_5'))
    model.add(Conv2D(112, (3,3), padding='same',activation='relu', name='conv_6'))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same', name='maxpool_3'))

    model.add(Flatten())
    model.add(Dense(448, activation='relu', name='dense_1'))
    model.add(Dropout(0.5))
    model.add(Dense(112, activation='relu', name='dense_2'))
   
    model.add(Activation('softmax'))
   

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    
    return model

In [ ]:
model = create_model_from_scratch()
History = model.fit(my_train,y_train,epochs=50,batch_size=28,validation_data = (my_val,y_val))

In [ ]:
from PIL import Image
import glob
X = []
L = []
for filename in glob.iglob(f'../input/ee379v-hw4/test/*.jpg'): #assuming gif
    name = os.path.split(filename)
    L.append(name[1])
    im=Image.open(filename)
    im = train_transform(im)
    X.append(np.array(im))


    
    
    
X=np.array(X)
X=X/255



In [ ]:
classes_x=np.argmax((model.predict(X) > 0.5).astype("int32"), axis = 1)
print(classes_x)

In [ ]:
import csv
with open('answers.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ID", "Category"])
    for i in range(len(classes_x)):
        writer.writerow([L[i], labels[classes_x[i]]])
    